In [1]:
import cv2
import traitlets
import numpy as np
from nano.camera import Camera
from ipywidgets import widgets

In [2]:
def bgr8_to_jpeg(value, quality=75):
    return bytes(cv2.imencode('.jpg', value)[1])

image = widgets.Image(format='jpeg', width=640, height=480)
print('Before instance()')
camera = Camera.instance()
print('After instance()')
camera_link = traitlets.dlink((camera, 'detected'), (image, 'value'), transform=bgr8_to_jpeg)
display(image)

Before instance()
After instance()


Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x02\x01\x0…

In [3]:
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util

In [4]:
import tensorflow as tf

In [5]:
PATH_TO_FROZEN_GRAPH='/home/singleye/projects/jetbot.singleye/object_detection/ssd_mobilenet_v1_coco_2017_11_17/frozen_inference_graph.pb'

detection_graph = tf.Graph()
with detection_graph.as_default():
  od_graph_def = tf.GraphDef()
  with tf.gfile.GFile(PATH_TO_FROZEN_GRAPH, 'rb') as fid:
    serialized_graph = fid.read()
    od_graph_def.ParseFromString(serialized_graph)
    tf.import_graph_def(od_graph_def, name='')

In [6]:
PATH_TO_LABELS='/home/singleye/projects/jetbot.singleye/object_detection/data/mscoco_label_map.pbtxt'
category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS, use_display_name=True)

In [7]:
def run_inference(camera, graph):
  with graph.as_default():
    with tf.Session() as sess:
      # Get handles to input and output tensors
      ops = tf.get_default_graph().get_operations()
      all_tensor_names = {output.name for op in ops for output in op.outputs}
      tensor_dict = {}
      for key in [
          'num_detections', 'detection_boxes', 'detection_scores',
          'detection_classes', 'detection_masks'
      ]:
        tensor_name = key + ':0'
        if tensor_name in all_tensor_names:
          tensor_dict[key] = tf.get_default_graph().get_tensor_by_name(
              tensor_name)
      if 'detection_masks' in tensor_dict:
        # The following processing is only for single image
        detection_boxes = tf.squeeze(tensor_dict['detection_boxes'], [0])
        detection_masks = tf.squeeze(tensor_dict['detection_masks'], [0])
        # Reframe is required to translate mask from box coordinates to image coordinates and fit the image size.
        real_num_detection = tf.cast(tensor_dict['num_detections'][0], tf.int32)
        detection_boxes = tf.slice(detection_boxes, [0, 0], [real_num_detection, -1])
        detection_masks = tf.slice(detection_masks, [0, 0, 0], [real_num_detection, -1, -1])
        detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(
            detection_masks, detection_boxes, image.shape[0], image.shape[1])
        detection_masks_reframed = tf.cast(
            tf.greater(detection_masks_reframed, 0.5), tf.uint8)
        # Follow the convention by adding back the batch dimension
        tensor_dict['detection_masks'] = tf.expand_dims(
            detection_masks_reframed, 0)
      image_tensor = tf.get_default_graph().get_tensor_by_name('image_tensor:0')

      while camera.running():
        image = camera.value.copy()
        # Run inference
        output_dict = sess.run(tensor_dict,
                                 feed_dict={image_tensor: np.expand_dims(image, 0)})
        camera.detected = vis_util.visualize_boxes_and_labels_on_image_array(image,
                                                                             output_dict['detection_boxes'][0],
                                                                             output_dict['detection_classes'][0].astype(np.uint8),
                                                                             output_dict['detection_scores'][0],
                                                                             category_index,
                                                                             instance_masks=None,
                                                                             use_normalized_coordinates=True,
                                                                             line_thickness=4)

In [8]:
run_inference(camera, detection_graph)

KeyboardInterrupt: 

In [9]:
camera.stop()